In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import SGDClassifier


def pegar_dataframe_por_num_resposta(num_resposta: int):
    return pd.read_csv('Perguntas 2018+ - {} respostas.csv'.format(num_respostas))

In [12]:
for num_respostas in range(3, 6):
    print("\n\n - - - - - Começando teste com {} respostas - - - - - \n\n".format(num_respostas))
    df = pegar_dataframe_por_num_resposta(num_respostas)

    # Filtrando apenas com perguntas com resposta escolhida
    df = df[df['alvo']!=-1]

    colunas_respostas = [ 'pontos_resposta', 'tempo_resposta', 'user_reputation', 'maestria_tags']
    colunas_relevantes = [c + str(resposta) for resposta in range(0, num_respostas) for c in colunas_respostas]

    x = df[colunas_relevantes]
    y = df['alvo']

    print('Tamanho dataset: ' + str(len(df)))

    # Preparamos o imputer
    # Utilizamos o fill value como -1.5, o que pareceu melhorar um pouco a previsão
    imputer = SimpleImputer(missing_values=np.NaN, strategy='constant', fill_value=-1.5)
    imputer.fit(x)
    x = imputer.transform(x)

    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15)

    # Realizamos Scaling
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)

    scaler = StandardScaler()
    scaler.fit(X_test)
    X_test = scaler.transform(X_test)

    print('Numero linhas Treino: ' + str(len(X_train)))
    print('Numero linhas Test: ' + str(len(X_test)))

    # Atualizado para utilizar
    model = SGDClassifier(loss='modified_huber', penalty='elasticnet')
    model.fit(X_train, y_train)


    y_pred_train = model.predict(X_train)

    cm_treinamento = confusion_matrix(y_train, y_pred_train)

    print('\nConjunto de treinamento:\n')
    print('Matriz Confusão:\n' + str(cm_treinamento) + "\n")
    print(classification_report(y_train, y_pred_train))

    y_pred = model.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)

    print('\nConjunto de testes:\n')
    print('Matriz Confusão:\n' + str(cm) + "\n")
    print(classification_report(y_test, y_pred))

    print("\n\n - - - - - Terminado teste com {} respostas - - - - - \n\n".format(num_respostas))




 - - - - - Começando teste com 3 respostas - - - - - 


Tamanho dataset: 750931
Numero linhas Treino: 638291
Numero linhas Test: 112640

Conjunto de treinamento:

Matriz Confusão:
[[143943  43202  25522]
 [ 25767 160416  26464]
 [ 38962  42495 131520]]

              precision    recall  f1-score   support

         0.0       0.69      0.68      0.68    212667
         1.0       0.65      0.75      0.70    212647
         2.0       0.72      0.62      0.66    212977

    accuracy                           0.68    638291
   macro avg       0.69      0.68      0.68    638291
weighted avg       0.69      0.68      0.68    638291


Conjunto de testes:

Matriz Confusão:
[[26383  6287  4752]
 [ 6245 25764  5636]
 [ 7011  6053 24509]]

              precision    recall  f1-score   support

         0.0       0.67      0.71      0.68     37422
         1.0       0.68      0.68      0.68     37645
         2.0       0.70      0.65      0.68     37573

    accuracy                           0.